In [1]:
%load_ext autoreload
%autoreload 2

from deep_nilmtk.disaggregator import NILMExperiment
from deep_nilmtk.models.pytorch import UNETNILM
from deep_nilmtk.data.loader.pytorch import GeneralDataLoader
from deep_nilmtk.utils.templates import ExperimentTemplate

import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from pathlib import Path


Seed set to 42


In [16]:
EXPERIMENT_NAME

'hipe_with_baselines_agg-ALL'

# HIPE

In [2]:
DATA_PATH = '../../data/'

## Test for all models

In [3]:
from deep_nilmtk.models.pytorch import UNETNILM, Seq2Point, Seq2Seq, SAED
from deep_nilmtk.utils.templates.baseline_templates.hipe_template_agg import hipe_aggregated_15min_agg, hipe_aggregated_5min_agg, hipe_aggregated_5min_agg_noise


#appliance = "oven_2"

#model = 'Seq2Pointbaseline'
# type = appliance.split('_')[0]
# instance = int(appliance.split('_')[1])

data_time_resolution = '5min_noise'

use_optuna = False
kfolds = 1
if not use_optuna:
    kfolds

if data_time_resolution == '5min':

    hipe_aggregated_5min_agg['experiment_settings']['use_optuna'] = use_optuna
    hipe_aggregated_5min_agg['experiment_settings']['kfolds'] = kfolds
    template_name = 'hipe_aggregated_5min_agg'
    RESULTS_PATH = '../../results/hipe/hipe_with_baselines_agg_5min'

elif data_time_resolution == '5min_noise':
    hipe_aggregated_5min_agg_noise['experiment_settings']['use_optuna'] = use_optuna
    hipe_aggregated_5min_agg_noise['experiment_settings']['kfolds'] = kfolds
    template_name = 'hipe_aggregated_5min_agg_noise'
    RESULTS_PATH = '../../results/hipe/hipe_with_baselines_agg_5min_noise'

else: # '15min':

    hipe_aggregated_15min_agg['experiment_settings']['use_optuna'] = use_optuna
    hipe_aggregated_15min_agg['experiment_settings']['kfolds'] = kfolds
    template_name = 'hipe_aggregated_15min_agg'
    RESULTS_PATH = '../../results/hipe/hipe_with_baselines_agg_15min'

list_appliances = [('motor',1),('motor',2), ('oven', 1), 'printer']
list_models = [('Seq2Pointbaseline', 'pytorch'), ('Seq2Seqbaseline', 'pytorch'), ('DAE', 'pytorch'), ('SAED_model', 'pytorch'),('BERT4NILM', 'pytorch'), ('UNET', 'pytorch'),('UNET_quantile', 'pytorch')]

# Cualquier parametro que querams modificar de get_template
model_config ={
    #'in_size': 121,
    #'out_size': sequence_length, # 1 or n according number of appliances
    'max_nb_epochs': 20,
    'n_trials': 20,
    'validation_metric': 'val_mae'
}


### Single Appliance

In [4]:
appliance = ('motor',2)
list_models = [('Seq2Pointbaseline', 'pytorch')] #('Seq2Pointbaseline', 'pytorch')]
model = list_models[0][0]

EXPERIMENT_NAME = f"hipe_with_baselines_agg-{appliance}-{model}"

template = ExperimentTemplate( data_path=DATA_PATH,
                template_name=template_name,
                list_appliances=[appliance], #(type,instance)],
                list_baselines_backends= list_models,
                model_config = model_config
                #  in_size=None,
                #  out_size=None,
                # max_nb_epochs=None
                )
template.run_template(EXPERIMENT_NAME,
                    RESULTS_PATH,
                    f'{RESULTS_PATH}/mlflow')

Joint Testing for all algorithms
Loading data for  hipe_agg  dataset
Dropping missing values
Generating predictions for : Seq2Pointbaseline
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed 2:   1%|          | 1/136 [00:00<00:04, 32.60it/s]

processed 136: 100%|██████████| 136/136 [00:01<00:00, 98.90it/s] 
............  mae  ..............
         Seq2Pointbaseline
motor 2           15.37466
............  nde  ..............
         Seq2Pointbaseline
motor 2           0.219295
............  rmse  ..............
         Seq2Pointbaseline
motor 2          59.931553
............  relative_error  ..............
         Seq2Pointbaseline
motor 2           0.094663
............  f1score  ..............
         Seq2Pointbaseline
motor 2           0.785981
Experiment took: 1.0 minutes


In [5]:
EXPERIMENT_NAME

"hipe_with_baselines_agg-('motor', 2)-Seq2Pointbaseline"

In [5]:
with open(f"../../results/hipe/hipe_with_baselines_agg_{data_time_resolution}/{EXPERIMENT_NAME}.p", 'rb') as f:
    results = pickle.load(f)
results.keys()

dict_keys(['error_keys', 'errors', 'train_mains', 'train_submeters', 'test_mains', 'test_submeters', 'gt', 'predictions', 'execution_time'])

In [6]:
wm_results=pd.DataFrame({
    key: results['predictions'][key].values.reshape(-1) for key in results['predictions']
}, index= results['test_submeters'][0][1][0].index)
wm_results['True consumption'] = results['test_submeters'][0][1][0].values

wm_results.head()

,Seq2Pointbaseline,True consumption
,,
2017-12-01 00:00:00+01:00,7.861515,0.0
2017-12-01 00:05:00+01:00,0.000000,0.0
2017-12-01 00:10:00+01:00,0.000000,0.0
2017-12-01 00:15:00+01:00,0.000000,0.0
2017-12-01 00:20:00+01:00,0.000000,0.0


In [ ]:
import plotly.express as px


if use_optuna:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{appliance}{training_type}")

### Multi Appliance-Models

In [4]:
EXPERIMENT_NAME = f"hipe_with_baselines_agg-ALL"

template = ExperimentTemplate( data_path=DATA_PATH,
                template_name=template_name,
                list_appliances=list_appliances, #[appliance], #(type,instance)],
                list_baselines_backends= list_models,
                model_config = model_config
                #  in_size=None,
                #  out_size=None,
                # max_nb_epochs=None
                )
template.run_template(EXPERIMENT_NAME,
                    RESULTS_PATH,
                    f'{RESULTS_PATH}/mlflow')

Joint Testing for all algorithms
Loading data for  hipe_agg  dataset
Dropping missing values
Generating predictions for : Seq2Pointbaseline
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed 1:   0%|          | 0/136 [00:00<?, ?it/s]

processed 136: 100%|██████████| 136/136 [00:01<00:00, 90.75it/s]


Generating predictions for : Seq2Seqbaseline
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed: 136: 100%|██████████| 136/136 [00:00<00:00, 137.92it/s]
(8542, 99, 1)
(8542, 99, 1)
(8542, 99, 1)
(8542, 99, 1)


Generating predictions for : DAE
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed: 136: 100%|██████████| 136/136 [00:00<00:00, 175.39it/s]
(8542, 1, 99)
(8542, 1, 99)
(8542, 1, 99)
(8542, 1, 99)


Generating predictions for : SAED_model
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed 136: 100%|██████████| 136/136 [00:02<00:00, 46.06it/s]


Generating predictions for : BERT4NILM
(8640, 1)
doing input normalisation on test data using main_params:
{'backend': 'pytorch', 'use_optuna': False, 'n_trials': 20, 'log_artificat': False, 'logs_path': 'logs', 'checkpoints_path': 'checkpoints', 'version': 0, 'max_nb_epochs': 20, 'batch_size': 64, 'clip_value': 10, 'learning_rate': 0.0001, 'patience_optim': 5, 'patience_check': 5, 'optimizer': 'adam', 'weight_decay': 0.0, 'momentum': None, 'decay_step': 100, 'gamma': 0.1, 'stride': 1, 'out_size': 1, 'in_size': 99, 'border': 0, 'multi_appliance': False, 'custom_preprocess': 'bert_preprocess', 'custom_postprocess': 'bert_postprocess', 'input_norm': 'z-norm', 'target_norm': 'z-norm', 'model_name': 'BERT4NILM', 'model_class': None, 'loader_class': <class 'deep_nilmtk.data.loader.pytorch.bert_dataloader.BERTDataset'>, 'seq_type': 'seq2point', 'point_position': 'mid_position', 'quantiles': [0.1, 0.25, 0.5, 0.75, 0.9], 'threshold_method': 'at', 'kfolds': 1, 'gap': 0, 'test_size': None, 'fea


Generating predictions for : UNET
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed: 136: 100%|██████████| 136/136 [00:01<00:00, 99.22it/s]


Generating predictions for : UNET_quantile
(8640, 1)
doing input normalisation on test data using main_params:
{'mean': array([[319.83711822]]), 'std': array([[880.45865654]])}
processed: 136: 100%|██████████| 136/136 [00:01<00:00, 91.95it/s]
............  mae  ..............
            Seq2Pointbaseline  Seq2Seqbaseline         DAE  SAED_model  \
(motor, 1)           9.890777        43.698040   19.647373   17.809008   
(motor, 2)          13.246608       224.646851  110.429893   27.494171   
(oven, 1)           59.616482       282.480621  202.422836   74.392921   
printer             16.262524        55.836823   38.910553   20.264656   

             BERT4NILM        UNET  UNET_quantile  
(motor, 1)   14.457746   23.859463      13.188826  
(motor, 2)  134.939041  117.772903      22.118183  
(oven, 1)   137.900146  241.527588     134.965240  
printer      29.144573   30.232635      14.186001  
............  nde  ..............
            Seq2Pointbaseline  Seq2Seqbaseline       DAE 

In [5]:
data_time_resolution

'5min_noise'

In [13]:
with open(f"../../results/hipe/hipe_with_baselines_agg_{data_time_resolution}/hipe_with_baselines_agg-ALL.p", 'rb') as f:
    results = pickle.load(f)
results.keys()

dict_keys(['error_keys', 'errors', 'train_mains', 'train_submeters', 'test_mains', 'test_submeters', 'gt', 'predictions', 'execution_time'])

In [14]:
#device_type = results['predictions']['Seq2Pointbaseline'].columns.get_level_values(0)[0]
device_instances = list(results['predictions']['Seq2Pointbaseline'].columns)

wm_results = []

for ind, instance in enumerate(device_instances): 

    wm_results.append(pd.DataFrame({
        #key: results['predictions'][key][device_type].values.reshape(-1) for key in results['predictions']
        key: results['predictions'][key][instance].values.reshape(-1) for key in results['predictions']
    }, index= results['test_submeters'][ind][1][0].index))
    wm_results[ind]['True consumption']= results['test_submeters'][ind][1][0].values


In [ ]:
import plotly.express as px


if use_optuna:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results[0]).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{training_type}-Chip Saw")

In [ ]:
import plotly.express as px


if hipe_aggregated_15min_agg['experiment_settings']['use_optuna']:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results[1]).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{training_type}-Vacuum Pump 1")

In [ ]:
import plotly.express as px


if hipe_aggregated_15min_agg['experiment_settings']['use_optuna']:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results[2]).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{training_type}-High Temperature Oven")

In [ ]:
import plotly.express as px


if hipe_aggregated_15min_agg['experiment_settings']['use_optuna']:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results[3]).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{training_type}-Printer")

In [ ]:
import plotly.express as px


if hipe_aggregated_15min_agg['experiment_settings']['use_optuna']:
    training_type="-HPO"
else: 
    training_type=""

fig = px.scatter(wm_results[3]).update_traces(mode="lines+markers")
fig.update_layout(title_text=f"{training_type}-Printer")

## Test for all appliances

In [ ]:
from deep_nilmtk.models.pytorch import UNETNILM, Seq2Point, Seq2Seq, SAED
from deep_nilmtk.utils.templates.baseline_templates.hipe_template import hipe_aggregated_15min

hipe_aggregated_15min['experiment_settings']['use_optuna'] = True

models = ['Seq2Pointbaseline', 'SAED_model', 'Seq2Seqbaseline', 'BERT4NILM', 'DAE', 'UNET', 'UNET_quantile']

# Cualquier parametro que querams modificar de get_template
model_config ={
    'in_size': 121,
    #'out_size': sequence_length, # 1 or n according number of appliances
    'max_nb_epochs': 10,
    'n_trials': 10
}

for model in models:

    EXPERIMENT_NAME = model
    
    template = ExperimentTemplate(data_path=DATA_PATH,
                    template_name='hipe_aggregated_15min',
                    list_appliances=[('motor',2), ('motor',4), ('motor',5), ('oven', 1), ('oven', 2), ('oven', 3),'printer','washing machine'],
                    list_baselines_backends=[(model, 'pytorch')], # ('DAE', 'pytorch')],#('Seq2Pointbaseline', 'pytorch'),('Seq2Pointbaseline', 'pytorch')], , ('UNET', 'pytorch'), ('UNET_quantile', 'pytorch')] ,#('Seq2Pointbaseline', 'pytorch')],#, ('SAED_model', 'pytorch'), ('unet_base', 'pytorch')], #], #models.py with get_template patrms , 
                    model_config = model_config
                    #  in_size=None,
                    #  out_size=None,
                    # max_nb_epochs=None
                    )
    
    template.run_template(EXPERIMENT_NAME,
                     RESULTS_PATH,
                     f'{RESULTS_PATH}/mlflow')

In [5]:
with open(f'{RESULTS_PATH}/{EXPERIMENT_NAME}.p', 'rb') as f:
    results = pickle.load(f)
results.keys()

dict_keys(['error_keys', 'errors', 'train_mains', 'train_submeters', 'test_mains', 'test_submeters', 'gt', 'predictions', 'execution_time'])

In [ ]:
wm_results=pd.DataFrame({
    key: results['predictions'][key].values.reshape(-1) for key in results['predictions']
}, index= results['test_submeters'][0][1][0].index)
wm_results['True consumption']= results['test_submeters'][0][1][0].values

wm_results

,Seq2Pointbaseline,True consumption
,,
2017-12-01 00:00:00+01:00,0.0,0.0
2017-12-01 00:15:00+01:00,0.0,0.0
2017-12-01 00:30:00+01:00,0.0,0.0
2017-12-01 00:45:00+01:00,0.0,0.0
2017-12-01 01:00:00+01:00,0.0,0.0
...,...,...
2017-12-30 22:45:00+01:00,0.0,0.0
2017-12-30 23:00:00+01:00,0.0,0.0
2017-12-30 23:15:00+01:00,0.0,0.0


## Template Extension

In [ ]:

# sequence_length = 121
# max_epochs = 10
optuna = False

seq2point_hpo = NILMExperiment({
                "model_class": Seq2Point,
                "loader_class": GeneralDataLoader,
                'model_name':'seq2point_base',
                'in_size': sequence_length,
                'custom_preprocess': None,
                #'out_size': 1, # 1 or n according number of appliances 
                'feature_type': 'mains', 
                'input_norm': 'z-norm', # get_template()
                'target_norm': 'z-norm', # get_template()
                #'seq_type': 'seq2point', # get_template()
                'point_position':'mid_position', # get_template()
                'learning_rate': 10e-5,  # get_template()
                'max_nb_epochs': max_epochs,
                'optuna': optuna,
                'n_trials': 50
            })

SAED_hpo = NILMExperiment({

    "model_class": SAED,        
    "loader_class": GeneralDataLoader,
    "model_name": 'saed_base',
    #'in_size': sequence_length,
    'custom_preprocess': None,
    'out_size': 1, # 1 or n according number of appliances 
    'feature_type': 'mains', # get_template() 
    'input_norm': 'z-norm', # get_template()
    'target_norm': 'z-norm', # get_template()
    #'seq_type': 'seq2point', # get_template()
    'point_position':'mid_position', # get_template()
    'attention_type': 'dot',
    'learning_rate': 10e-5,  # get_template()¡
    'max_nb_epochs': max_epochs,
    'optuna':optuna,
    'n_trials': 50

}) 

"""

seq2seq_base = NILMExperiment({
                "model_class": Seq2Seq,
                "loader_class": GeneralDataLoader,
                'model_name':'seq2seq_base',
                #'in_size': sequence_length,
                'custom_preprocess': None,
                #'out_size': 128 # 1*params['n_zize'] or n*in_size according number of appliances 
                #'feature_type': 'mains', # get_template()
                #'input_norm': 'z-norm', # get_template()
                #'target_norm': 'z-norm', # get_template()
                #'seq_type': 'seq2seq', # get_template()
                #'learning_rate': 10e-5,  # get_template()
                #'max_nb_epochs': max_epochs,
                'optuna':optuna
            })

unet_base = NILMExperiment({
    
    "model_class": UNETNILM,        
    "loader_class": GeneralDataLoader,
    "model_name": 'unet_base',
    #'in_size': sequence_length,
    'custom_preprocess': None,
    #'out_size': 1, # 1 or n according number of appliances 
    #'feature_type': 'mains', # get_template()
    #'input_norm': 'z-norm', # get_template()
    #'target_norm': 'z-norm', # get_template()
    #'seq_type': 'seq2seq', # get_template()
    #'learning_rate': 10e-5,  # get_template()
    #'max_nb_epochs': max_epochs,
    'optuna':optuna

})
"""

'\n\nseq2seq_base = NILMExperiment({\n                "model_class": Seq2Seq,\n                "loader_class": GeneralDataLoader,\n                \'model_name\':\'seq2seq_base\',\n                #\'in_size\': sequence_length,\n                \'custom_preprocess\': None,\n                #\'out_size\': 128 # 1*params[\'n_zize\'] or n*in_size according number of appliances \n                #\'feature_type\': \'mains\', # get_template()\n                #\'input_norm\': \'z-norm\', # get_template()\n                #\'target_norm\': \'z-norm\', # get_template()\n                #\'seq_type\': \'seq2seq\', # get_template()\n                #\'learning_rate\': 10e-5,  # get_template()\n                #\'max_nb_epochs\': max_epochs,\n                \'optuna\':optuna\n            })\n\nunet_base = NILMExperiment({\n    \n    "model_class": UNETNILM,        \n    "loader_class": GeneralDataLoader,\n    "model_name": \'unet_base\',\n    #\'in_size\': sequence_length,\n    \'custom_preproc

In [ ]:
template.extend_experiment({'seq2point_hpo': seq2point_hpo, 'SAED_hpo': SAED_hpo}) #, 'SAED_base': SAED_base, 'seq2seq_base': seq2seq_base, 'unet_base': unet_base,})